# Otázky

1. když provedu `single_correction`, mám vrátit tu korekci a pak teprve
 updatovat hyperparametry? Dává to smysl, jinak by vyšlo několikrát to samé...

In [1]:
import numpy as np
import common
from exp_family import update_IW, init_IW_hyp, get_IW_pars_from_hyp
import exp_family

In [2]:
def single_correction(measurement):
    """
    Performs a single correction based on a single measurement.
    """
    return xikk

#master = Master(nodes)
#...
def node_loop():
    measurement = perform_measurement()
    y, P, R, hyp_P, hyp_R = single_correction(measurement)
    new_hyp_P, new_hyp_R = master.update_with_isclose(self) #master.update_with_neighbourghs()

In [3]:
from typing import Tuple
from collections import defaultdict

In [4]:
Matrix = np.ndarray
MultiShape = np.ndarray
Scalar = np.number
Vector = np.ndarray

class IWPrior:
    hp: MultiShape = None
    p: Scalar = None
    
    def __init__(self, nu: Scalar, psi: Matrix):
        self.p = psi.shape[0]
        self.hp = np.array([
            -0.5 * psi, 
            -0.5 * (nu + self.p + 1)
        ])
    
    @property
    def psi(self) -> Matrix:
        return -2 * self.hp[0]
    
    @property
    def nu(self) -> Scalar:
        return -2 * self.hp[1] - self.p - 1
    
    def expect(self):
        return self.psi / (self.nu - self.p - 1)
    
class MeasurementNode:
    P_prior: IWPrior = None
    R_prior: IWPrior = None
    F: Matrix = None
    Q: Matrix = None
    last_state: Vector = None
        
    logger = defaultdict(list)
    
    def __init__(self,
                 x: Vector,
                 P_prior: IWPrior,
                 R_prior: IWPrior,
                 F: Matrix,
                 Q: Matrix,
                 H: Matrix,
                 rho: Scalar,
                 tau: Scalar,
                 N = 10,
                ):
        self.last_state = x
        self.P_prior = P_prior
        self.R_prior = R_prior
        self.F = F
        self.Q = Q
        self.H = H
        self.rho = rho
        self.tau = tau
        
        self.P = self.P_prior.expect()
        
    def predict_state(self) -> Tuple[Vector, Matrix]:
        return common.time_update(self.last_state, self.P, self.F, self.Q)
    
    def single_update(self, state_prediction, P_prediction, measurement,
                     init_hyp_P, init_hyp_R):
        x = state_prediction
        P = P_prediction
        for i in range(N):
            R, hyp_R = update_IW(init_hyp_R, 
                                 measurement, 
                                 self.H @ x, 
                                 self.H @ P @ self.H.T)
            P, hyp_P = update_IW(init_hyp_P, x, state_prediction, P)
            P, x = common.kalman_correction(H, P, R, state_prediction, measurement)
                        
        self.last_state = x
        self.P = P
        self.P_prior.hp = hyp_P
        self.R_prior.hp = hyp_R
        return x, P
    
    def _init_hyp_P(self):
        init = exp_family.init_P_hyp(self.tau, self.P)
        return self.P_prior.hp + init
    
    
    def _init_hyp_R(self):
        init = exp_family.init_R_hyp(self.rho, self.R_prior.psi, self.R_prior.nu)
        return self.R_prior.hp + init
    
    
    def single_kf(self, measurement):
        x_predicted, P_predicted = self.predict_state()
        init_hyp_P = self._init_hyp_P()
        init_hyp_R = self._init_hyp_R()
        return self.single_update(x_predicted, P_predicted, measurement,
                                  init_hyp_P, init_hyp_R)
        
    def log(self, key, val):
        self.logger[key].append(val)

In [5]:
def node_factory(x, P, u, U, F, Q, H, rho, tau):
    P_p = IWPrior(P.shape[0] + tau + 1, tau * P)
    R_p = IWPrior(u, U)    
    return MeasurementNode(x, P_p, R_p, F, Q, H, rho, tau)

In [6]:
traj, xk, P, tau, rho, u, U, H, F, Q, N = common.init_all()

In [7]:
m1 = node_factory(xk, P, u, U, F, Q, H, rho, tau)
m2 = node_factory(xk, P, u, U, F, Q, H, rho, tau)

In [15]:
m1.predict_state()

(array([-0.17392499,  1.21671945,  1.        ,  1.        ]),
 array([[ 1.28334377e+00, -1.47633243e-05,  2.50774573e+00,
         -9.82710936e-12],
        [-1.47633243e-05,  1.28339498e+00, -9.82710744e-12,
          2.50774573e+00],
        [ 2.50774573e+00, -9.82710744e-12,  2.52774573e+01,
         -9.93629648e-11],
        [-9.82710936e-12,  2.50774573e+00, -9.93629648e-11,
          2.52774573e+01]]))

In [16]:
y = traj.Y.T[0]

In [17]:
#%pdb

In [18]:
m1.single_kf(y)

(array([-0.27553119,  1.11511578,  1.        ,  1.        ]),
 array([[ 6.33458074e-01, -6.02265309e-06, -3.55636296e-14,
          6.69548919e-14],
        [-6.02265309e-06,  6.33478236e-01,  6.69556057e-14,
         -1.87121905e-13],
        [-3.55636296e-14,  6.69556057e-14,  2.19158552e+01,
         -8.75333527e-11],
        [ 6.69548919e-14, -1.87121905e-13, -8.75333527e-11,
          2.19158552e+01]]))

In [20]:
t = 0
x_log = []
P_log = []
for zk in traj.Y.T:
    #print('===== Step: ', t, '======') 
    t += 1
    
    #### Time update
    # xkl
    xk = common.predict_state(F, xk)  # xll
    # Pkl
    P = common.predict_PECM(F, P, Q)  # Pll
    
    #### Measurement update
    ## Initialization - step 3
    # Pikk
    Pik = P
    # xikk
    xikk = xk
    
    # tkl, Tkl, ukl, Ukl 
    #_, _, ukl, Ukl = common.init(P, tau, P.shape[0], rho, u, U.shape[0], U)
    
    #hyp_P = np.array(init_IW_hyp(Tkk, tkk))
    #print(hyp_P)
    hyp_P = np.array([
        -0.5 * tau * P,
        -0.5 * tau - 1 - P.shape[0]
    ])
    #print(np.isclose(hyp_P[1], new_p[1]))#.all().all())
    hyp_P_prev = hyp_P.copy()
    
    hyp_R = np.array([
        rho * hyp_R[0],
        -0.5 * ((rho * (-2 * hyp_R[1] - hyp_R[0].shape[0] - 1) ) + hyp_R[0].shape[0] + 1)
    ])

    #hyp_R = init_IW_hyp(Ukl, ukl)
    hyp_R_prev = hyp_R.copy()
    #print(hyp_R[0], new_r[0])
    #print(np.isclose(hyp_R[0], new_r[0]).all().all())
    #print(hyp_R[1] - new_r[1])
    
    ## END of Initialization - step 3
    
    ## VB iters
    for i in range(N):
        Rjk, hyp_R = update_IW(hyp_R_prev, zk, H @ xikk, H @ Pik @ H.T)
        Pjkl, hyp_P = update_IW(hyp_P_prev, xikk, xk, Pik)
        # Pjkk, xjkk
        Pik, xikk = common.kalman_correction(H, Pjkl, Rjk, xk, zk)
    
    # Assignment - step 13
    # xkk
    xk = xikk
    # Pkk
    P = Pik
    # ukk, Ukk
    U, u = get_IW_pars_from_hyp(hyp_R)
    
    x_log.append(xk)
    P_log.append(P)
    
x_log = np.array(x_log).squeeze().T
P_log = np.array(P_log).squeeze()

In [21]:
# KF, just measurements
np.sqrt(((x_log[:2] - traj.X[:2]) ** 2 ).mean()), np.sqrt(((traj.Y[:2] - traj.X[:2]) ** 2 ).mean())

(0.4608634981693899, 0.7668096325216082)

In [22]:
# velocity: KF
np.sqrt(((x_log[2:] - traj.X[2:]) ** 2 ).mean())

1.2353960171632807

In [ ]:
common.plot_results(traj, x_log)